In [80]:
from pathlib import Path

import geopandas as gpd
import httpx
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.csv
import pytz
import shapely
from shapely.geometry import box, shape
from ipywidgets import FloatRangeSlider, dlink, jsdlink
from matplotlib.colors import LogNorm, Normalize
from palettable.colorbrewer.diverging import BrBG_10
from palettable.colorbrewer.sequential import Oranges_9
from sidecar import Sidecar

from lonboard import Map, SolidPolygonLayer,viz
from lonboard.colormap import apply_continuous_cmap
from lonboard.layer_extension import DataFilterExtension
from lonboard.controls import MultiRangeSlider

In [37]:
vector_endpoint = "https://eoapi.ifrc-risk.k8s.labs.ds.io/vector"
response = httpx.get(f"{vector_endpoint}/collections").json()
collections = response["collections"]
for collection in collections:
    print(collection["id"])

public.population_hexbins_afghanistan
public.health_facilities_afg_osm
public.admin_boundaries_usa_adm0
public.admin_boundaries_usa_adm1
public.earthquake_usgs_gov_shakemap_afg_pga
public.earthquake_usgs_gov_shakemap_afg_psa1p0
public.admin_boundaries_usa_adm2
public.earthquake_usgs_gov_shakemap_afg_pgv
public.earthquake_usgs_gov_shakemap_afg_psa3p0
public.earthquake_usgs_gov_shakemap_afg_psa0p3
public.earthquake_usgs_gov_shakemap_afg_mi
public.postgis_srs
public.postgis_srs_all
public.postgis_srs_search
public.st_subdivide
public.st_hexagongrid
public.st_squaregrid


In [38]:
def get_all_items(url, limit=100, bbox=None):
    all_items = []

    while True:
        params = {"limit": limit, "bbox": bbox}
        response = httpx.get(url, params=params, follow_redirects=True)
        if response.status_code == 200:
            items = response.json()
            all_items.extend(items.get("features", []))
            next_link = next((link["href"] for link in items.get("links", []) if link.get("rel") == "next"), None)
            if next_link:
                url = next_link
            else:
                break
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")
            print(response.text)
            break

    return all_items

In [39]:
admin_2_url = f"{vector_endpoint}/collections/public.admin_boundaries_usa_adm2/items"

In [40]:
admin_2_items = get_all_items(admin_2_url) 
for item in admin_2_items:
    item['geometry'] = shape(item['geometry'])

admin_2_gdf = gpd.GeoDataFrame(admin_2_items)
admin_2_gdf

,type,geometry,id,properties,links
0,Feature,"MULTIPOLYGON (((-179.12890 51.21250, -179.1295...",0106000020E6100000EC00000001030000000100000005...,"{'adm0_iso': 'USA', 'adm1_name': 'Alaska', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."
1,Feature,"MULTIPOLYGON (((-81.27140 31.30970, -81.27060 ...",0106000020E61000001400000001030000000100000006...,"{'adm0_iso': 'USA', 'adm1_name': 'Georgia', 'a...","[{'title': 'Collection', 'href': 'http://eoapi..."
2,Feature,"MULTIPOLYGON (((-81.47640 31.03700, -81.47330 ...",0106000020E61000001000000001030000000100000006...,"{'adm0_iso': 'USA', 'adm1_name': 'Georgia', 'a...","[{'title': 'Collection', 'href': 'http://eoapi..."
3,Feature,"MULTIPOLYGON (((-82.13260 31.47160, -82.13280 ...",0106000020E610000001000000010300000001000000C5...,"{'adm0_iso': 'USA', 'adm1_name': 'Georgia', 'a...","[{'title': 'Collection', 'href': 'http://eoapi..."
4,Feature,"MULTIPOLYGON (((-81.22110 31.68170, -81.22440 ...",0106000020E61000000600000001030000000100000005...,"{'adm0_iso': 'USA', 'adm1_name': 'Georgia', 'a...","[{'title': 'Collection', 'href': 'http://eoapi..."
...,...,...,...,...,...
3143,Feature,"MULTIPOLYGON (((-169.27660 63.16820, -169.2792...",0106000020E6100000A00000000103000000010000000A...,"{'adm0_iso': 'USA', 'adm1_name': 'Alaska', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."
3144,Feature,"MULTIPOLYGON (((-161.79790 66.03130, -161.7995...",0106000020E61000006700000001030000000100000005...,"{'adm0_iso': 'USA', 'adm1_name': 'Alaska', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."
3145,Feature,"MULTIPOLYGON (((-166.20630 68.26880, -166.2099...",0106000020E61000003701000001030000000100000006...,"{'adm0_iso': 'USA', 'adm1_name': 'Alaska', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."
3146,Feature,"MULTIPOLYGON (((-141.00300 68.43930, -141.0069...",0106000020E610000001000000010300000001000000DB...,"{'adm0_iso': 'USA', 'adm1_name': 'Alaska', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."


In [49]:
fill_rgb = [204, 251, 254] # light blue
border_rgb = [37, 36, 34] # dark border color


# visualize with lonboard
layer = SolidPolygonLayer.from_geopandas(
    admin_2_gdf,
    auto_highlight=True,
    extruded=True,
    filled=False,
    pickable=True,
    get_fill_color=fill_rgb,
    get_line_color=border_rgb,
    wireframe=True,
)

map_ = Map(layers=[layer])

In [51]:
hawai_admin = admin_2_gdf[admin_2_gdf['properties'].apply(lambda x: x.get('adm1_name') == 'Hawaii')]
hawai_admin

,type,geometry,id,properties,links
3138,Feature,"MULTIPOLYGON (((-156.91450 21.16670, -156.9202...",0106000020E61000000300000001030000000100000034...,"{'adm0_iso': 'USA', 'adm1_name': 'Hawaii', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."
3139,Feature,"MULTIPOLYGON (((-157.65780 21.32020, -157.6566...",0106000020E61000000C00000001030000000100000005...,"{'adm0_iso': 'USA', 'adm1_name': 'Hawaii', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."
3140,Feature,"MULTIPOLYGON (((-156.49530 20.63000, -156.4976...",0106000020E61000000900000001030000000100000007...,"{'adm0_iso': 'USA', 'adm1_name': 'Hawaii', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."
3141,Feature,"MULTIPOLYGON (((-155.87190 20.08830, -155.8800...",0106000020E61000000100000001030000000100000071...,"{'adm0_iso': 'USA', 'adm1_name': 'Hawaii', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."
3142,Feature,"MULTIPOLYGON (((-160.08560 22.00280, -160.0855...",0106000020E6100000060000000103000000010000006F...,"{'adm0_iso': 'USA', 'adm1_name': 'Hawaii', 'ad...","[{'title': 'Collection', 'href': 'http://eoapi..."


In [65]:
# A GeoDataFrame with Polygon or MultiPolygon geometries
hawai_layer = SolidPolygonLayer.from_geopandas(
    hawai_admin,
    get_fill_color=[255, 0, 0],
)
m = Map(layers=[hawai_layer, layer])

In [66]:
sidecar = Sidecar()
with sidecar:
    display(m)

In [67]:
stac_endpoint = "https://eoapi.ifrc-risk.k8s.labs.ds.io/stac"


In [93]:
hawaii_bbox = hawai_admin.total_bounds
hawaii_bbox_geom = box(*hawaii_bbox)

intersecting_collections = []

for collection in collections:
    for bbox in collection["extent"]["spatial"]["bbox"]:
        collection_bbox_geom = box(*bbox)  
        if collection_bbox_geom.intersects(hawaii_bbox_geom) and collection["id"] not in [x["id"] for x in intersecting_collections]:
            print(collection["id"])
            intersecting_collections.append(collection)


SyntaxError: '(' was never closed (2093075318.py, line 10)